# Data manipulation

## Preparing the dataset

In [1]:
# defining path and CMU Book Summary Dataset dataset URL
DATA_DIR = '../data/'
DS_URL = 'https://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz'

In [2]:
# downloading the CMU-BSD
import urllib.request
import tarfile
import json
import os

os.chdir(DATA_DIR)
urllib.request.urlretrieve(DS_URL, 'booksummaries.tar.gz')
targzf = tarfile.open('booksummaries.tar.gz')
targzf.extractall('.')
targzf.close()
os.remove('booksummaries.tar.gz')

In [4]:
# loading the downloaded dataset
os.chdir('booksummaries')
txtfile = open('booksummaries.txt')

wiki_ids, frbs_ids, names, authors, dates, genres, summaries = [], [], [], [], [], [], []

for line in txtfile.readlines():
    entry = line.split('\t')
    wiki_ids.append(entry[0])
    frbs_ids.append(entry[1])
    names.append(entry[2])
    authors.append(entry[3])
    dates.append(entry[4])
    if entry[5].strip() != '': genres.append(list(json.loads(entry[5]).values()))
    else: genres.append([])
    summaries.append(entry[6].strip())

txtfile.close()

In [5]:
# conversion to csv
import pandas as pd
import shutil

dataset = pd.DataFrame({'wiki_id': wiki_ids, 'frbs_id': frbs_ids, 'name': names,
                        'author': authors, 'date': dates, 'genres': genres, 'summary': summaries})

os.chdir('..')
dataset.to_csv('dataset.csv')
shutil.rmtree("booksummaries")